# Chain of Responsibility: Method Chain

> Chaining methods one after another

In this scenario we're designing a videogame with creatures roaming about and attacking each other and the player. These creatures will have some base stats for attack and defense, which can also be boosted with powerup modifiers, like picking up a sword or something.

Let's implement our creature.

In [1]:
class Creature:
    def __init__(self, name, attack, defense):
        self.defense = defense
        self.attack = attack
        self.name = name

    def __str__(self):
        return f'{self.name} ({self.attack}/{self.defense})'

Let's now implement the modifiers. We'll create a base modifyer class that we will use for all of our modifiers.

In [ ]:
class CreatureModifier:
    def __init__(self, creature):
        self.creature = creature
        self.next_modifier = None # 1

    def add_modifier(self, modifier): # 2
        if self.next_modifier:
            self.next_modifier.add_modifier(modifier)
        else:
            self.next_modifier = modifier

    def handle(self): # 3
        if self.next_modifier:
            self.next_modifier.handle()

There are many things happening in this class:
1. A creature can have multiple modifyers applied to it at once (like attack boost and defense debuff, for example), and these modifyers will most likely happen one after the another, so what we do is create a **modifyer chain**: the `next_modifier` attribute is a pointer to a method that we will call after this one, thus `next_modifier` is implementing a **Method Chain**.
2. `add_modifier` is where we chain our modifier methods: if we don't have a modifier yet, we assign it to `next_modifier`, but if we already do have one, we call `add_modifier` on our stored modifier and pass it the next modifier, thus creating our **Method Chain**.
3. `handle` is where we apply our **chain of responsibility** to the creature.: we recursively call `handle` on `next_modifier` so that all of the modifiers can be run. Note that `CreatureModifying` is a base class, so it's up to its implementations to add whatever action the modifier is supposed to do.

Now that we have our base modifier class in place, let's actually create a modifier:

In [5]:
class DoubleAttackModifier(CreatureModifier):
    def handle(self):
        print(f'Doubling {self.creature.name}\'s attack')
        self.creature.attack *= 2
        super().handle() # This line is important!

Note that our modifier is implementing the `handle` method and, most importantly, is calling `super().handle()` after it does its thing, because the base class' `handle` is the one in charge of propagating the chain of responsibility and gets the other modifiers of the creature, if there are any, to work as well.

Let's see it in action:

In [6]:
goblin = Creature('Goblin', 1, 1)
print(goblin)

root = CreatureModifier(goblin)
root.add_modifier(DoubleAttackModifier(goblin))
root.handle() # apply modifiers
print(goblin)

Goblin (1/1)
Doubling Goblin's attack
Goblin (2/1)


We instantiate the base `CreatureModifier` class and chain modifiers to it, and then call `handle` to apply the modifiers in order. We can even apply the same modifier multiple times:

In [7]:
spider = Creature('Spider', 1, 2)
print(spider)

root = CreatureModifier(spider)
root.add_modifier(DoubleAttackModifier(spider))
root.add_modifier(DoubleAttackModifier(spider))
root.add_modifier(DoubleAttackModifier(spider))
root.handle() # apply modifiers
print(spider)

Spider (1/2)
Doubling Spider's attack
Doubling Spider's attack
Doubling Spider's attack
Spider (8/2)


Let's create additional modifyers and combine them:

In [8]:
class IncreaseDefenseModifier(CreatureModifier):
    def handle(self):
        if self.creature.attack <= 2:
            print(f'Increasing {self.creature.name}''s defense')
            self.creature.defense += 1
        super().handle()

class NoBonusesModifier(CreatureModifier):
    def handle(self):
        print('No bonuses for you!')
        # We're deliberately omitting the important line!

The `NoBonusesModifier` is a special modifier that blocks the creature from gaining additional modifiers, maybe because it got poisoned or some other status condition. We implement it by simply omitting the `super().handle()` line, which stops the chain of responsibility from propagating.

Let's combine our modifiers and see how they work:

In [15]:
goblin = Creature('Goblin', 1, 1)
print(goblin)

root = CreatureModifier(goblin)

root.add_modifier(IncreaseDefenseModifier(goblin)) # (1/2)
root.add_modifier(DoubleAttackModifier(goblin)) # (2/2)
root.add_modifier(DoubleAttackModifier(goblin)) # (4/2)
root.add_modifier(IncreaseDefenseModifier(goblin)) # no effect! Our attack is too powerful
root.add_modifier(NoBonusesModifier(goblin)) # no more bonuses from now on
root.add_modifier(DoubleAttackModifier(goblin)) # no effect! Otherwise it would be (8/2)

root.handle()  # apply modifiers
print(goblin)

Goblin (1/1)
Increasing Goblins defense
Doubling Goblin's attack
Doubling Goblin's attack
No bonuses for you!
Goblin (4/2)
